In [ ]:
def is_inside(points, target):
    num_intersects = 0
    point_inf = (float('inf'), target.y)
    points.append(points[0])
    for i in range (0, len(points) - 1):
        point1 = points[i]
        point2 = points[i+1]
        if do_intersect(point1, point2, target, point_inf):	
            num_intersects += 1
    return num_intersects % 2 != 0

def do_intersect(pointA, pointB, pointC, pointD):
    a1 = pointB[1] - pointA[1]
    b1 = pointA[0] - pointB[0]
    c1 = a1 * (pointA[0]) + b1 * (pointA[1])
    a2 = pointD[1] - pointC[1]
    b2 = pointC[0] - pointD[0]
    c2 = a2 * (pointC[0]) + b2 * (pointC[1])
    determinant = a1 * b2 - a2 * b1
    if (determinant == 0):
        return False
    else:
        x = (c1*b2 - c2*b1)/determinant
        y = (a1*c2 - a2*c1)/determinant
        isValidX = min(pointA[0], pointB[0]) <= x <= max(pointA[0], pointB[0]) 
        isValidY = min(pointA[1], pointB[1]) <= y <= max(pointA[1], pointB[1]) 

        return isValidX and isValidY